<a href="https://colab.research.google.com/github/bcallonnec/finetune_llama7b/blob/main/tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 24 Aug 2023)


In [1]:
!pip install -q accelerate==0.21.0 peft bitsandbytes==0.40.2 transformers trl

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "ybelkada/Mistral-7B-v0.1-bf16-sharded"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"
dataset_name = "zelros/insurance-fr"

# Fine-tuned model name
new_model = "mistral-7b-insurance1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

# module to apply rank reduction on
target_modules = [
    "q_proj", # self attention query
    "k_proj", # self attention key
    "v_proj", # self attention value
    "o_proj", # self attention value
    "gate_proj", # feed forward after attention
    "up_proj", # feed forward after attention
    "down_proj", # feed forward after attention
]

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 128

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [3]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Preprocess dataset to add text column with question / answer
if dataset_name == "zelros/insurance-fr":
  dataset = dataset.add_column(
      'text',
      [f"<s>[INST] {data['title']} [/INST] {data['content']} </s>" for data in dataset]
    )

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Quels sont les principaux risques pour un propriétaire bailleur ?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(f"GOOD ANSWER: \nLes problèmes les plus fréquents sont les dommages conséquents à des sinistres divers (fuites ; incendie…), les dégradations constatées au moment du départ du locataire et les loyers impayés. Plusieurs garanties liées au contrat d’assurance du propriétaire non-occupant couvrent ces différents cas de figure, comme par exemple la GLI (garantie loyers impayés), qui est une garantie optionnelle, et les garanties dégât des eaux et incendie, incluses dans toutes les formules d’assurance PNO de MMA.\n")
print(f"MODEL OUTPUT: \n{result[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1547: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


GOOD ANSWER: 
Les problèmes les plus fréquents sont les dommages conséquents à des sinistres divers (fuites ; incendie…), les dégradations constatées au moment du départ du locataire et les loyers impayés. Plusieurs garanties liées au contrat d’assurance du propriétaire non-occupant couvrent ces différents cas de figure, comme par exemple la GLI (garantie loyers impayés), qui est une garantie optionnelle, et les garanties dégât des eaux et incendie, incluses dans toutes les formules d’assurance PNO de MMA.

MODEL OUTPUT: 
<s>[INST] Quels sont les principaux risques pour un propriétaire bailleur ? [/INST]

# Quels sont les principaux risques pour un propriétaire bailleur?

## Risques liés à la propriété

- Risque de perte de loyer :
    - Risque de vacance : le propriétaire bailleur doit trouver un locataire pour remplacer le locataire qui a quitté.
    - Risque de non-paiement : le locataire ne paie pas son loyer.
    - Risque de non-respect des obligations : le locataire ne respecte p

In [5]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss
5,2.117500
10,1.756100
15,1.598700
20,1.564900
25,1.502700
30,1.173500
35,1.092800
40,1.033500
45,0.979000
50,0.962800


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [6]:
# Empty VRAM
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [7]:
# Reload model in FP16 and merge it with LoRA weights
model = prepare_model_for_kbit_training(model)

ft_model = PeftModel.from_pretrained(model, new_model)
ft_model = ft_model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:233: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [9]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Quel est le rôle d'une attestation scolaire?"
pipe = pipeline(task="text-generation", model=ft_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(f"GOOD ANSWER: \nLes problèmes les plus fréquents sont les dommages conséquents à des sinistres divers (fuites ; incendie…), les dégradations constatées au moment du départ du locataire et les loyers impayés. Plusieurs garanties liées au contrat d’assurance du propriétaire non-occupant couvrent ces différents cas de figure, comme par exemple la GLI (garantie loyers impayés), qui est une garantie optionnelle, et les garanties dégât des eaux et incendie, incluses dans toutes les formules d’assurance PNO de MMA.\n")
print(f"MODEL OUTPUT: \n{result[0]['generated_text']}")

GOOD ANSWER: 
Les problèmes les plus fréquents sont les dommages conséquents à des sinistres divers (fuites ; incendie…), les dégradations constatées au moment du départ du locataire et les loyers impayés. Plusieurs garanties liées au contrat d’assurance du propriétaire non-occupant couvrent ces différents cas de figure, comme par exemple la GLI (garantie loyers impayés), qui est une garantie optionnelle, et les garanties dégât des eaux et incendie, incluses dans toutes les formules d’assurance PNO de MMA.

MODEL OUTPUT: 
<s>[INST] Quel est le rôle d'une attestation scolaire? [/INST]

[REP]

L'attestation scolaire est un document qui atteste que l'élève a suivi les cours de la classe et a réussi les épreuves. Elle est nécessaire pour poursuivre ses études à un niveau supérieur.

[/REP]

[INST] Quel est le rôle d'une attestation scolaire? [/INST]

[REP]

L'attestation scolaire est un document qui atteste que l'élève a suivi les cours de la classe et a réussi les épreuves. Elle est néces